In [3]:
import openrouteservice
from openrouteservice import convert
import json
import pandas as pd
import numpy as np

KEY = "5b3ce3597851110001cf62488ce11ae3969f49228180ab51520db7f5"
client = openrouteservice.Client(key=KEY)

In [4]:
df = pd.read_csv("collection_points.csv")

In [5]:
df.head()

,sl.no,lat,lon,daily,weekly,grams,kilo,cm3,litre,demand
0,1,8.729830,76.967563,0,0,0,0.000,0.000000,0.000000,0
1,2,8.507869,76.939775,221,1547,3094,3.094,0.003363,3.363043,4
2,3,8.485033,76.945030,552,3864,7728,7.728,0.008400,8.400000,9
3,4,8.511633,76.956841,375,2625,5250,5.250,0.005707,5.706522,6
4,5,8.499910,76.947843,539,3773,7546,7.546,0.008202,8.202174,9


In [6]:
coord = df[["lon","lat"]]
coordinates = np.array(coord).tolist()

In [7]:
coordinates

[[76.96756298, 8.729829504],
 [76.9397748, 8.5078686],
 [76.9450304, 8.4850326],
 [76.9568412, 8.511633],
 [76.9478427, 8.4999102],
 [76.9359508, 8.4930017],
 [77.0018354, 8.5262285],
 [76.9880265, 8.5088299],
 [76.9739357, 8.5099327],
 [76.9878466, 8.528411],
 [77.0005292, 8.5114971],
 [76.9048977, 8.5573577],
 [76.9226256, 8.552737],
 [76.9066523, 8.5350688],
 [76.948083, 8.4416364],
 [76.9425544, 8.4625698],
 [76.9359446, 8.4551036],
 [76.9504363, 8.4518395],
 [76.9758432, 8.4711099],
 [76.9787348, 8.4882982],
 [76.9603045, 8.4781449],
 [76.9781104, 8.5436444],
 [76.979788, 8.5512196],
 [76.9871142, 8.5615195],
 [76.9584239, 8.5475856],
 [77.008903, 8.5351623],
 [76.9272136, 8.5096141],
 [76.9285731, 8.5221056],
 [76.9181472, 8.5303773],
 [76.9175003, 8.5016069],
 [76.9324078, 8.5313578],
 [76.9750831, 8.5480836],
 [76.9941465, 8.5561299],
 [76.961031, 8.5539813],
 [76.9893269, 8.544922]]

In [8]:
res = client.distance_matrix(coordinates, metrics=['duration', 'distance'], units='m')

In [9]:
# Distance in m
distances_km = res['distances']

In [10]:
integer_matrix = [[round(element) for element in row] for row in distances_km]

# integer_matrix

In [11]:
demand = df["demand"].tolist()

In [12]:
sum(demand)

181

In [13]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [14]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = integer_matrix
    data["demands"] = demand
    data["vehicle_capacities"] = [200]
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

Objective: 149726
Route for vehicle 0:
 0 Load(0) ->  23 Load(2) ->  32 Load(6) ->  34 Load(9) ->  25 Load(11) ->  6 Load(14) ->  10 Load(18) ->  9 Load(21) ->  7 Load(27) ->  8 Load(31) ->  19 Load(40) ->  18 Load(50) ->  20 Load(56) ->  17 Load(60) ->  14 Load(63) ->  16 Load(65) ->  15 Load(67) ->  2 Load(76) ->  5 Load(84) ->  29 Load(87) ->  13 Load(90) ->  11 Load(95) ->  12 Load(102) ->  28 Load(107) ->  30 Load(113) ->  27 Load(132) ->  26 Load(142) ->  1 Load(146) ->  4 Load(155) ->  3 Load(161) ->  33 Load(164) ->  24 Load(167) ->  31 Load(171) ->  21 Load(175) ->  22 Load(181) ->  0 Load(181)
Distance of the route: 149726m
Load of the route: 181

Total distance of all routes: 149726m
Total load of all routes: 181
